In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd
import seaborn as sns

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "end_to_end_project"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# for pretty printing
def printDf(sprkDF): 
    newdf = sprkDF.toPandas()
    from IPython.display import display, HTML
    return HTML(newdf.to_html())

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

# Spark libs
from pyspark.sql.session import SparkSession

# helper functions
from helpers.helper_functions import translate_to_file_string

In [ ]:
from pyspark.sql.session import SparkSession
from pyspark.sql.types import BooleanType
from pyspark.sql.functions import expr
from pyspark.ml.stat import Summarizer
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StringIndexer, StandardScaler
from helpers.helper_functions import translate_to_file_string

In [ ]:
inputFile = translate_to_file_string("../data/Flight_Delay_Jan_2020_ontime.csv")

In [ ]:
spark = (SparkSession
       .builder
       .appName("FlightDataStatistics")
       .getOrCreate())

In [ ]:
pysparkDF = spark.read.option("header", "true") \
        .option("inferSchema", "true") \
        .option("delimiter", ",") \
        .csv(inputFile) \
        
pysparkDF.printSchema()

In [ ]:
# Number of columns
anzahlSpalten = len(pysparkDF.columns)
print("Der Datensatz enthält " + str(anzahlSpalten) + " Spalten")

In [ ]:
# Number of rows
anzahlZeilen = pysparkDF.count()
print("Der Datensatz enthält " + str(anzahlZeilen) + " Zeilen")

In [ ]:
# Check whether OP_CARRIER_FL_NUM is merely running id for flights or rather encoding specific trips (e.g. Istanbul -> New York)
pysparkDF.groupby('OP_CARRIER_FL_NUM').count().show()

In [ ]:
# A tail number refers to an identification number painted on an aircraft, frequently on the tail.
# Check amount of flights per plane
pysparkDF.groupby('TAIL_NUM').count().show()

# Check average flights per plane per year
pysparkDF.groupby('TAIL_NUM').count().agg(F.mean('count')).show()

In [ ]:
# Check whether ORIGIN_AIRPORT_ID is 1:1 mapping onto ORIGIN
pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN']).count().count() == pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN']).count().dropDuplicates(['ORIGIN_AIRPORT_ID']).count()

# -> ORIGIN_AIRPORT_ID is string indexing ORIGIN

In [ ]:
# Check whether ORIGIN_AIRPORT_ID is 1:1 mapping onto ORIGIN_AIRPORT_SEQ_ID
pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID']).count().count() == pysparkDF.groupby(['ORIGIN_AIRPORT_ID','ORIGIN_AIRPORT_SEQ_ID']).count().dropDuplicates(['ORIGIN_AIRPORT_ID']).count()

# -> ORIGIN_AIRPORT_ID is 1:1 mapping to ORIGIN_AIRPORT_SEQ_ID

In [ ]:
# Check whether DEST_AIRPORT_ID is 1:1 mapping onto DEST
pysparkDF.groupby(['DEST_AIRPORT_ID','DEST']).count().count() == pysparkDF.groupby(['DEST_AIRPORT_ID','DEST']).count().dropDuplicates(['DEST_AIRPORT_ID']).count()

# -> DEST_AIRPORT_ID is string indexing DEST

In [ ]:
# Check whether DEST_AIRPORT_ID is 1:1 mapping onto DEST_AIRPORT_SEQ_ID
pysparkDF.groupby(['DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID']).count().count() == pysparkDF.groupby(['DEST_AIRPORT_ID','DEST_AIRPORT_SEQ_ID']).count().dropDuplicates(['DEST_AIRPORT_SEQ_ID']).count()

# -> DEST_AIRPORT_ID is 1:1 mapping to ORIGIN_AIRPORT_SEQ_ID

In [ ]:
# Check whether OP_UNIQUE_CARRIER is 1:1 mapping onto OP_CARRIER
pysparkDF.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER']).count().count() == pysparkDF.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER']).count().dropDuplicates(['OP_UNIQUE_CARRIER']).count()

# -> OP_UNIQUE_CARRIER is 1:1 mapping to OP_CARRIER

In [ ]:
# Check whether TAIL_NUM is 1:1 mapping onto OP_CARRIER_FL_NUM
pysparkDF.groupby(['TAIL_NUM','OP_CARRIER_FL_NUM']).count().count() == pysparkDF.groupby(['TAIL_NUM','OP_CARRIER_FL_NUM']).count().dropDuplicates(['OP_CARRIER_FL_NUM']).count()

# -> TAIL_NUM is not 1:1 mapping to OP_CARRIER_FL_NUM -> One distinct plane can fly multiple routes

In [ ]:
# Transform Dataframe to Pandas 
pandasDF =pysparkDF.toPandas()

In [ ]:
# Excerpt from the dataset
pandasDF.head(10)

In [ ]:
# Key figures
pandasDF.describe()

In [ ]:
# Number of flights delayed and on time 
print("Anzahl an pünktlichen (0) und verspäteten (1) Flügen beim Abflug")
print(pandasDF['DEP_DEL15'].value_counts())

print("Anzahl an pünktlichen (0) und verspäteten (1) Flügen bei der Ankunft")
print(pandasDF['ARR_DEL15'].value_counts())

In [ ]:
# Show cancelled, diverted and delayed flight counts and total distance per Carrier in a table
groupCarrier = pandasDF.groupby("OP_CARRIER")
groupCarrier[['CANCELLED', "DIVERTED",'ARR_DEL15', 'DEP_DEL15', 'DISTANCE']].sum().sort_values(["CANCELLED"], ascending=False)

In [ ]:
# Show cancelled, diverted and delayed flight counts and total distance per origin airport in a table
groupDeprature = pandasDF.groupby("ORIGIN")
groupDeprature[['CANCELLED', "DIVERTED",'DEP_DEL15', 'DISTANCE']].sum().sort_values(["DISTANCE"], ascending=False)

### Data Visualization

Histograms

In [ ]:
pysparkDF.drop('OP_CARRIER','DEST','DEST_AIRPORT_SEQ_ID','OP_UNIQUE_CARRIER', 'ORIGIN_AIRPORT_SEQ_ID', 'ORIGIN', '_c21', 'TAIL_NUM', 'OP_CARRIER_FL_NUM').toPandas().hist(bins=50, figsize=(20,15))
save_fig("attribute_histogram_plots")
plt.show()

Stacked Bar

In [ ]:
# Show Count of flights delayed and on time

# Create bars for filtered value delayed (1) and on time (0)
ax = pysparkDF.groupby(["OP_CARRIER","DEP_DEL15"]).count().filter(pysparkDF.DEP_DEL15 == 0).toPandas().sort_values(by="OP_CARRIER",ascending=True).plot.bar(stacked=True, x="OP_CARRIER", color='Blue')
pysparkDF.groupby(["OP_CARRIER","DEP_DEL15"]).count().filter(pysparkDF.DEP_DEL15 == 1).toPandas().sort_values(by="OP_CARRIER",ascending=True).plot.bar(stacked=True, x="OP_CARRIER", color='Orange', ax=ax )


plt.ylabel("Count Flight")

save_fig("stacked_bar")
plt.show()

Grouped Bar Chart

In [ ]:
# Show count of cancelled, diverted and delayed flights per airline
pandasDF.groupby(by="OP_CARRIER")[['OP_CARRIER','CANCELLED', "DIVERTED",'ARR_DEL15']].sum().plot.bar()

save_fig("grouped_bar")
plt.show()

Bar Chart 100%

In [ ]:
# Create bar chart with normalized values
import matplotlib.patches as mpatches

Data = pd.read_csv('../data/Flight_Delay_Jan_2020_ontime.csv')
Data.head()

x,y = 'DAY_OF_WEEK', 'DEP_DEL15'

#Normalize Counts
(Data
.groupby(x)[y]
.value_counts(normalize=True)
.mul(100)
.rename('percentage')
.reset_index()
.pipe((sns.catplot,'data'), x= 'DAY_OF_WEEK',y='percentage',hue=y, kind='bar'))


save_fig("normalized_bar")
# show the graph
plt.show()



Pie Chart

In [ ]:
#Pie Chart - Show Cancelled Flights as pecentage per day of week
pysparkDF.groupby("DAY_OF_WEEK", "CANCELLED").count().filter(pysparkDF.CANCELLED==1).toPandas().plot.pie(y="DAY_OF_WEEK", autopct="%.1f%%", legend=False)
plt.suptitle("Cancelled Flights")
save_fig("pie_chart")
plt.show()


Correlation

In [ ]:
# Show Correlation of attributes
import matplotlib.patches as mpatches

plt.figure(figsize = (12, 10))
sns.heatmap(pandasDF.corr(), annot = True, cmap = 'vlag')
save_fig("correlation_heatmap")
plt.show()

Scatter Plot

In [ ]:
# Scatter Plot for delayed flights 
ax = pysparkDF.select("OP_CARRIER_AIRLINE_ID", "ORIGIN_AIRPORT_ID").filter(pysparkDF.ARR_DEL15 != 0).toPandas().plot.scatter(x='OP_CARRIER_AIRLINE_ID', y='ORIGIN_AIRPORT_ID', color='DarkBlue', label='ARR_DEL15 1')
pysparkDF.select("OP_CARRIER_AIRLINE_ID","ORIGIN_AIRPORT_ID").filter(pysparkDF.DEP_DEL15 != 0).toPandas().plot.scatter(x='OP_CARRIER_AIRLINE_ID', y='ORIGIN_AIRPORT_ID', color='Yellow', label='DEP_DEL15 1', ax=ax)

save_fig("scatter_plots")
plt.show()

Scatter Matrix

In [ ]:
# Scatter Matrix for multiple columns
from pandas.plotting import scatter_matrix
scatter_matrix(pysparkDF.select("OP_CARRIER_AIRLINE_ID", "ORIGIN_AIRPORT_ID", "DEST_AIRPORT_ID", "DEP_DEL15", "Distance").toPandas(), alpha=0.2, figsize=(30, 30), diagonal='kde')
save_fig("scatter_matrix")
plt.show()

In [ ]:
#spark.stop()